In [11]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

import sshtunnel
import psycopg2 as pg
import json
import os

import duckdb

In [12]:
def read_ccw_json(ccw_json, condition):
    with open(ccw_json, 'r') as json_file:
        ccw_dict = json.load(json_file)
    
    diag_string = (
        ",".join([f"'{x}'" for x in ccw_dict[condition]["icd9"]]) + 
        "," +
        ",".join([f"'{x}'" for x in ccw_dict[condition]["icd10"]])
    )

    ref_period = ccw_dict[condition]["ref_period"]

    return ref_period, diag_string

def get_years_before_ref_year(ref_year, first_year):
    return [year for year in range(first_year, ref_year + 1)]

In [13]:
dw_adm_prefix = "../data/input/local_data/data_warehouse/dw_adm_xu_sabath_00_16/adm"

## start connection ----
conn = duckdb.connect()

ref_period, diag_string = read_ccw_json("../data/input/remote_data/ccw.json", "alzh")

In [14]:
print(ref_period)
print(diag_string)

3
'3310','G300','G301','G308','G309'


In [15]:
%%time

diag_files = [f"{dw_adm_prefix}_{year}.parquet" for year in get_years_before_ref_year(2005, 2000)]
diag_list = []
for file in diag_files:
    diag = conn.execute(f"""
        SELECT DISTINCT
            bene_id,
            admission_date, 
            diag 
        FROM '{file}', UNNEST(diagnoses) AS adm(diag)
        WHERE adm.diag IN ({diag_string})
    """).fetchdf()
    diag_list.append(diag)

## Preparing adm ----


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [17]:
len(diag_list)

6

In [18]:
diag = pd.concat(diag_list)
diag.shape

(2274243, 3)

In [22]:
diag = diag[['bene_id', 'admission_date']].groupby('bene_id').min().reset_index()
diag.rename({'admission_date': 'first_adm_date'}, axis=1, inplace=True)

In [24]:
print(diag.shape)
print(diag.columns)

(1380620, 2)
Index(['bene_id', 'first_adm_date'], dtype='object')
